In [4]:
#미세먼지
import pandas as pd
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [19]:
with open('combined_dust.csv', 'r') as f:
    dust = pd.read_csv(f)
len(dust)

83780

In [20]:
dust.head()

,시간,지점,1시간평균 미세먼지농도(㎍/㎥)
0,2013-03-19 01:00:00,156,80.0
1,2013-03-19 02:00:00,156,89.0
2,2013-03-19 03:00:00,156,79.0
3,2013-03-19 04:00:00,156,80.0
4,2013-03-19 05:00:00,156,89.0


In [21]:
import pandas as pd

# '시간' 칼럼을 datetime 유형으로 변환
dust['시간'] = pd.to_datetime(dust['시간'])

# 2013년부터 2022년까지의 완전한 날짜 및 시간 범위 생성
full_date_range = pd.date_range(start='2013-03-19 01:00', end='2022-12-31 23:00', freq='H')

# 누락된 날짜 및 시간 찾기
missing_dates = full_date_range[~full_date_range.isin(dust['시간'])]

print(missing_dates)
print(len(missing_dates))

DatetimeIndex(['2013-04-04 07:00:00', '2013-04-04 09:00:00',
               '2013-04-04 10:00:00', '2013-04-04 11:00:00',
               '2013-04-04 12:00:00', '2013-04-04 13:00:00',
               '2013-04-04 14:00:00', '2013-04-24 06:00:00',
               '2013-04-24 08:00:00', '2013-04-24 09:00:00',
               ...
               '2022-08-03 17:00:00', '2022-08-03 18:00:00',
               '2022-10-04 08:00:00', '2022-10-04 09:00:00',
               '2022-10-04 10:00:00', '2022-10-04 11:00:00',
               '2022-10-04 12:00:00', '2022-10-04 13:00:00',
               '2022-11-03 14:00:00', '2022-12-20 18:00:00'],
              dtype='datetime64[ns]', length=2020, freq=None)
2020


In [22]:
dust.reset_index(inplace=True)
duplicated_times = dust[dust['시간'].duplicated()]
print(duplicated_times)

       index         시간   지점  1시간평균 미세먼지농도(㎍/㎥)
32215  32215 2017-01-01  156               44.0


In [23]:
index_to_check = 32215
rows_around_duplicated = dust.iloc[index_to_check-1:index_to_check+2]
print(rows_around_duplicated)

       index                  시간   지점  1시간평균 미세먼지농도(㎍/㎥)
32214  32214 2017-01-01 00:00:00  156               44.0
32215  32215 2017-01-01 00:00:00  156               44.0
32216  32216 2017-01-01 01:00:00  156               56.0


In [24]:
# 중복된 행 제거
dust.drop(32215, inplace=True)

# 다시 중복된 '시간' 값이 있는지 확인
duplicated_times = dust[dust['시간'].duplicated()]
print(duplicated_times)

Empty DataFrame
Columns: [index, 시간, 지점, 1시간평균 미세먼지농도(㎍/㎥)]
Index: []


In [25]:
# '시간' 열을 인덱스로 설정합니다.
dust.set_index('시간', inplace=True)

# 전체 날짜 및 시간 범위 생성
full_date_range = pd.date_range(start='2013-03-19 01:00', end='2022-12-31 23:00', freq='H')

# 새로운 인덱스를 사용하여 데이터프레임을 다시 인덱싱
dust = dust.reindex(full_date_range)

# 인덱스 이름 재설정
dust.index.name = '시간'

# 인덱스를 다시 열로 변경
dust.reset_index(inplace=True)
dust.head()

,시간,index,지점,1시간평균 미세먼지농도(㎍/㎥)
0,2013-03-19 01:00:00,0.0,156.0,80.0
1,2013-03-19 02:00:00,1.0,156.0,89.0
2,2013-03-19 03:00:00,2.0,156.0,79.0
3,2013-03-19 04:00:00,3.0,156.0,80.0
4,2013-03-19 05:00:00,4.0,156.0,89.0


In [29]:
len(dust)

85799

In [ ]:
# '시간' 열을 인덱스로 설정합니다.
dust.set_index('시간', inplace=True)

# 시간 기반으로 보간을 진행합니다.
dust.interpolate(method='time', inplace=True)

# 보간 후 누락된 값이 있는지 확인합니다.
missing_values_after_interpolation = dust[dust['1시간평균 미세먼지농도(㎍/㎥)'].isna()]
print(missing_values_after_interpolation)

In [30]:
# '시간' 인덱스를 다시 열로 설정
dust.reset_index(inplace=True)

# '시간' 칼럼을 복사하여 '시간_복사' 칼럼에 저장
dust['시간_복사'] = dust['시간']

# 다시 '시간' 열을 DatetimeIndex로 설정
dust.set_index('시간', inplace=True)

# 보간 전 누락된 값의 인덱스를 저장합니다.
missing_indices_before = dust[dust['1시간평균 미세먼지농도(㎍/㎥)'].isna()].index

# 시간 기반으로 '1시간평균 미세먼지농도(㎍/㎥)' 시리즈만 보간합니다.
dust['1시간평균 미세먼지농도(㎍/㎥)'] = dust['1시간평균 미세먼지농도(㎍/㎥)'].interpolate(method='time')


In [31]:
dust.head()

,index,지점,1시간평균 미세먼지농도(㎍/㎥),시간_복사
시간,,,,
2013-03-19 01:00:00,0.0,156.0,80.0,2013-03-19 01:00:00
2013-03-19 02:00:00,1.0,156.0,89.0,2013-03-19 02:00:00
2013-03-19 03:00:00,2.0,156.0,79.0,2013-03-19 03:00:00
2013-03-19 04:00:00,3.0,156.0,80.0,2013-03-19 04:00:00
2013-03-19 05:00:00,4.0,156.0,89.0,2013-03-19 05:00:00


In [15]:
for idx in missing_indices_before[:10]:
    surrounding_data = dust.loc[idx-pd.Timedelta(hours=1):idx+pd.Timedelta(hours=1), '1시간평균 미세먼지농도(㎍/㎥)']
    print(surrounding_data)
    print('-' * 50)  # 구분선 추가

시간
2013-04-04 06:00:00    62.727273
2013-04-04 07:00:00    61.454545
2013-04-04 08:00:00    60.181818
Name: 1시간평균 미세먼지농도(㎍/㎥), dtype: float64
--------------------------------------------------
시간
2013-04-04 08:00:00    60.181818
2013-04-04 09:00:00    58.909091
2013-04-04 10:00:00    57.636364
Name: 1시간평균 미세먼지농도(㎍/㎥), dtype: float64
--------------------------------------------------
시간
2013-04-04 09:00:00    58.909091
2013-04-04 10:00:00    57.636364
2013-04-04 11:00:00    56.363636
Name: 1시간평균 미세먼지농도(㎍/㎥), dtype: float64
--------------------------------------------------
시간
2013-04-04 10:00:00    57.636364
2013-04-04 11:00:00    56.363636
2013-04-04 12:00:00    55.090909
Name: 1시간평균 미세먼지농도(㎍/㎥), dtype: float64
--------------------------------------------------
시간
2013-04-04 11:00:00    56.363636
2013-04-04 12:00:00    55.090909
2013-04-04 13:00:00    53.818182
Name: 1시간평균 미세먼지농도(㎍/㎥), dtype: float64
--------------------------------------------------
시간
2013-04-04 12:00:00    55.090909

In [32]:
dust = dust.drop(columns=['index', '지점'])
dust.head()

,1시간평균 미세먼지농도(㎍/㎥),시간_복사
시간,,
2013-03-19 01:00:00,80.0,2013-03-19 01:00:00
2013-03-19 02:00:00,89.0,2013-03-19 02:00:00
2013-03-19 03:00:00,79.0,2013-03-19 03:00:00
2013-03-19 04:00:00,80.0,2013-03-19 04:00:00
2013-03-19 05:00:00,89.0,2013-03-19 05:00:00


In [33]:
dust.to_csv('dust.csv', index=False, encoding='cp949')

In [34]:
with open('dust.csv', 'r') as f:
    dust_new = pd.read_csv(f)
len(dust_new)

85799

In [39]:
dust_new.head()

,aerosol,time_copy
0,80.0,2013-03-19 01:00:00
1,89.0,2013-03-19 02:00:00
2,79.0,2013-03-19 03:00:00
3,80.0,2013-03-19 04:00:00
4,89.0,2013-03-19 05:00:00


In [38]:
dust_new = dust_new.rename(columns={
    '1시간평균 미세먼지농도(㎍/㎥)': 'aerosol',
    '시간_복사': 'time_copy',
    '시간': 'time'
})

In [40]:
dust_new.to_csv('dust.csv', index=False, encoding='cp949')

In [41]:
with open('dust.csv', 'r') as f:
    dust_new = pd.read_csv(f)
dust_new.head()

,aerosol,time_copy
0,80.0,2013-03-19 01:00:00
1,89.0,2013-03-19 02:00:00
2,79.0,2013-03-19 03:00:00
3,80.0,2013-03-19 04:00:00
4,89.0,2013-03-19 05:00:00
